<a href="https://colab.research.google.com/github/avtokit2700/nlp/blob/master/ULMFit_pretained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from IPython.display import display, HTML, IFrame
display(HTML('<h1>Hello Google Colab</h1>'))

# Import requierments libraries

In [0]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html 

!pip install fastai

# import libraries 
import fastai 
from fastai import * 
from fastai.text import * 
import pandas as pd 
import numpy as np 
from functools import partial 
import io 
import os

In [4]:
from sklearn.datasets import fetch_20newsgroups 

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove= 
                             ('headers', 'footers', 'quotes'))

documents = dataset.data

df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})

print(df.shape)
df = df[df['label'].isin([1,10])] 
df = df.reset_index(drop = True)
df['label'].value_counts()

(11314, 2)


10    600
1     584
Name: label, dtype: int64

# Explore dataset

In [5]:
df.head()

,label,text
0,10,"Well, I will have to change the scoring on my ..."
1,1,Archive-name: graphics/resources-list/part1\nL...
2,10,"\nAnd of course, Mike Ramsey was (at one time)..."
3,10,"As I promised, I would give you the name of th..."
4,10,GAME(S) OF 4/15\n---------------\nADIRONDACK 6...


In [7]:
df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")
import nltk
nltk.download('stopwords') 
from nltk.corpus import stopwords 

stop_words = stopwords.words('english')

# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split()) 

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x:[item for item in x if 
                                    item not in stop_words]) 


# de-tokenization 
detokenized_doc = [] 

for i in range(len(df)):
    t =' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['text'] = detokenized_doc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
from sklearn.model_selection import train_test_split 

# split data into training and validation set 
df_trn, df_val = train_test_split(df, stratify = df['label'], 
                                  test_size = 0.4, 
                                  random_state = 12)

df_trn.shape, df_val.shape

((710, 2), (474, 2))

In [0]:
# Language model data 
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = 
                                  df_val, path = "") 

# Classifier model data 
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, 
                                      valid_df = df_val,  
                                      vocab=data_lm.train_ds.vocab, 
                                      bs=32)

In [10]:
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103,  
                               drop_mult=0.7)
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)
learn.save_encoder('ft_enc')

epoch,train_loss,valid_loss,accuracy
1,7.785500,6.203495,0.144438


In [11]:
learn = text_classifier_learner(data_clas, drop_mult=0.7)
learn.load_encoder('ft_enc')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy


KeyboardInterrupt: ignored

In [0]:
# get predictions 
preds, targets = learn.get_preds() 
predictions = np.argmax(preds, axis = 1) 

pd.crosstab(predictions, targets)